In [6]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import scipy as sp
import math as m
import pickle as pk
import resources.RES as res
# import resources.res.store_interdependencies
from ladybug.sunpath import Sunpath as sp
from sklearn.preprocessing import minmax_scale as sk_minmax
import pymorton as pm
import pygmo as pg

In [7]:
# import lattice
env_lat_path = os.path.relpath("../data/macrovoxels.csv")
envelope_lattice = tg.lattice_from_csv(env_lat_path)

# plot dimensions - USER INPUT
plot_area = 6000

# required FSI - USER INPUT
FSI = 3

area_req = FSI * plot_area
voxs_req = int(area_req / (envelope_lattice.unit[0] * envelope_lattice.unit[1]))

# number of variables:
num_var = envelope_lattice.flatten().shape[0]

# actual PV sun obstructing cost array:
c1_norm = pk.load(open("../data/c1_norm.pk", "rb"))

# actual daylighting obstructing cost array:
c2_norm = pk.load(open("../data/c2_norm.pk", "rb"))

# actual sky view factor obstructing cost array:
c4_norm = pk.load(open("../data/c4_norm.pk", "rb"))

In [37]:
class test_python:

    # Number of dimensions
    def __init__(self,dim):
        self.dim = dim

    # Define objectives    
    def fitness(self, x):
        f1 = 0 # direct radiation on voxel roofs/PV potential (TODO: split faces and roofs, take internals/neighbour blocks into account since the blocked voxels never contribute)
        for i in range(num_var):
            f1 -= x[i] * c1_norm[i] #/sum(c1_norm) # We need to maximize; pygmo minimizes by default
        f2 = 0 # daylighting potential of voxels
        for j in range(num_var):
            f2 -= x[j] * c2_norm[j] #/sum(c2_norm) # We need to maximize; pygmo minimizes by default
        f3 = -(1 - (abs(voxs_req - sum(x)))/voxs_req) # FSI
        f4 = 0
        for k in range(num_var):
            f4 -= x[k] * c4_norm[k] #/sum(c4_norm)
        # TODO: f5 compactness
        return [f1, f2, f3, f4]
    
    # Return number of objectives
    def get_nobj(self):
        return 4

    # Return bounds of decision variables
    def get_bounds(self):
        return (np.full((self.dim,),0.),np.full((self.dim,),1.))

    # return number of integer variables (all variables are integer in this case TODO: transparency vectors for smoother shapes/results)
    def get_nix(self):
        return self.dim

    # Return function name
    def get_name(self):
        return "Test function MAX no.1"

In [38]:
# create User Defined Problem
prob = pg.problem(test_python(dim = num_var))

In [39]:
# create population
pop = pg.population(prob, size=8)

# select algorithm
algo = pg.algorithm(pg.nsga2(gen=150))

# run optimization
pop = algo.evolve(pop)

# extract results
fits, vectors = pop.get_f(), pop.get_x()

# extract and print non-dominated fronts
ndf, dl, dc, ndr = pg.fast_non_dominated_sorting(fits)

# ax = pg.plot_non_dominated_fronts(pop.get_f()) # plotting the non dominated fronts #TODO: what exactly does this mean in this context

In [40]:
best = pg.sort_population_mo(points = pop.get_f())[0] # the best solutions (by population)

In [41]:
print("The best configuration is: \n", pop.get_x()[best], "\n It's fitness is: ", pop.get_f()[best], "\n This is population #", best)

The best configuration is: 
 [1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 1. 0. 1. 0.] 
 It's fitness is:  [-35.25926703 -44.95322696  -0.675      -38.38931298] 
 This is population # 0


In [42]:
np.count_nonzero(pop.get_x()[best]) # TODO: does the FSI fitness requirement constrain the model too much? 
# TODO: should the other fitness functions be normalized (again) so that all objectives can achieve a maximum of -1? --> increases computation time without changing results

106

In [43]:
def reshape_and_store_to_lattice(values_list, envelope_lattice):
    env_all_vox_id = envelope_lattice.indices.flatten()
    env_all_vox = envelope_lattice.flatten() # envelope inclusion condition: True-False
    env_in_vox_id = env_all_vox_id[env_all_vox] # keep in-envelope voxels (True)

    # initialize array
    values_array = np.full(env_all_vox.shape, 0.0)
    
    # store values for the in-envelope voxels
    values_array[env_in_vox_id] = values_list

    # reshape to lattice shape
    values_array_3d = values_array.reshape(envelope_lattice.shape)

    # convert to lattice
    values_lattice = tg.to_lattice(values_array_3d, envelope_lattice)

    return values_lattice

In [44]:
configuration = reshape_and_store_to_lattice(pop.get_x()[best], envelope_lattice)

In [45]:
# visualize configuration 
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
configuration.fast_vis(p)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(204.88887394336027, 84.88887394336027, 144.88887394336027),
 (60.0, -60.0, 0.0),
 (0.0, 0.0, 1.0)]